# landscape classification

In this case study, we will be exploring the use of convolution neural networks to classify images of landscapes. Identifying landsacpe or vegetation or land-use types from image data has important applications in agriculture and natural-resource management.

In this example, we'll be using a data set of images from the following possible types or classes:

* buildings representing cities or other human habitations
* forests
* glacier or ice-covered landscapes
* mountains
* sea or ocean
* streets or paved areas

I've packaged the image data into separate training and validation sub-sets, available for downoad at zenodo.

The following code cell will download the image data sets, confirm the number of images in each sub-set, and package the data sets into tensorflow Dataset objects for neural network training.

In [ ]:
import tensorflow as tf
import pathlib

# download training data
train_url = 'https://zenodo.org/record/5512793/files/train.tgz'
train_dir = tf.keras.utils.get_file(origin=train_url, fname='train', untar=True)
train_dir = pathlib.Path(train_dir)

# download validation data
valid_url = 'https://zenodo.org/record/5512793/files/valid.tgz'
valid_dir = tf.keras.utils.get_file(origin=valid_url, fname='valid', untar=True)
valid_dir = pathlib.Path(valid_dir)

# print number of training and validation images
train_image_count = len(list(train_dir.glob('*/*.jpg')))
valid_image_count = len(list(valid_dir.glob('*/*.jpg')))
print(train_image_count, valid_image_count)

# package images into tensorflow dataset objects
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)
# print tensorflow dataset objects
print(train_data, valid_data)

It should take a few seconds to download the data sets.

You should see that there are 14,034 total image files in the training data sub-set, and 3000 images in the validation sub-set. In both cases, there are 6 possible classes or landscape types.

Notice that the shape of the training and validation Dataset objects is the same:

    ((None, 150, 150, 3), (None, ))

That is, the image data (ignoring the batch dimension of None) consists of 150x150 pixel images with 3 color channels (ie, typical RGB image data). The labels are integer-valued class labels, which is pretty standard for image classification problems.

We'll need to remember that the images are 150x150x3, so we can specify the correct input shape for our neural network.

Also, we'll need to use SparseCategoricalCrossentropy loss when we fit our model to the training data, because the category labels are not one-hot encoded.

To start off, we'll build a very simple convolution neural network consisting of a single convolution layer with a single 3x3 filter and ReLU activation.

As a 'trick', we're going to automatically rescale our image data to be on the [0,1] scale *automatically* in our neural network. We'll do this by specifying a tf.keras.layers.experimental.preprocessing.Rescaling layer as the first layer in the network.

Because our image data has pixel values between 0 and 255, we'll need to 'rescale' them by a factor of:

    1.0/255

which we specify as the scaling factor for the Rescaling layer. We'll also need to specify the input shape of the network when we create the Rescaling layer, because it's the first layer in the network.

After 'flattening' the output of the convolution layer, we create a Dense output layer with 6 units (because there are 6 possible landscape classes), and softmax activation.

We're going to opt for the Adam optimizer in this case, as it will help our model fit run faster (ie, fewer epochs). With this much data, we don't want to wait around for the slower SGD optimzer to reach a good model fit. The Adam optimizer is typically 'faster' than SGD, and it has been widely used for training image classification networks.

Make sure we specify SparseCategoricalCrossentropy loss, record the model's accuracy as it trains, and we'll train for 20 epochs.

Make sure you use GPU resources for this run, or it will take a *long* time! Click on the downward-facing arrow in the upper right corner of colab, select "View resources", and then click "Change Runtime Type". Select "GPU" from the "Hardware acceleration" drop-down, and save. Now your model fit will run on a GPU.

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

This model has 131,458 trainable parameters, nearly all of them in the Dense output layer.

You'll notice that, with the Adam optimizer, the model reaches ~0.99 accuracy on the *training* data after only a few epochs of training. But, the accuracy on the *validation* data stays *very* low (around 0.38 in my case)!

There is clearly an 'overfitting' problem. This makes sense, given that we have 14,034 training images and 131,458 model parameters!

Let's try adding a Dropout layer to reduce overfitting.

In the following code cell, we remove 90% of the outputs from the convolution layer, before flattening the data and sending it to the Dense output layer.

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Well, we appear to have alleviated model overfitting to the training data; the model's accuracy on the training and validation data sub-sets is much more similar.

But, accuracy is pretty *low*, overall. In my case, I achieved a final model accuracy of 0.49 on the training data and 0.50 on the validation data. So, about 50% of the images are being correctly classified, but at least our model isn't overfitting.

Let's see if we can improve model accuracy, without exacerbating overfitting.

XX - bigger network.